Declaración de librerías

In [1]:
import os
#import kagglehub
import pandas as pd

Lectura de archivo

In [2]:
#https://ourworldindata.org/covid-cases
df = pd.read_csv('owid-covid-data.csv')
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN


In [ ]:
df.columns

Contagios vs Muertes : Joel Ramírez

In [3]:
# Copia independiente del DataFrame original
conf_deaths_df = df.copy()

# Convertir a datetime
conf_deaths_df.loc[:, 'date_format'] = pd.to_datetime(conf_deaths_df['date'])

# Crear una nueva columna para el mes utilizando una estrategia diferente
conf_deaths_df.loc[:, 'month'] = conf_deaths_df['date_format'].dt.to_period('M')

# Agrupar por mes y sumar los valores de 'new_cases' y 'new_deaths'
grouped_month_df = conf_deaths_df.groupby('month').agg({'new_cases': 'sum', 'new_deaths': 'sum', 'new_vaccinations' : 'sum'})
# Mostrar las primeras filas del DataFrame agrupado por mes
grouped_month_df.head(15)


,new_cases,new_deaths,new_vaccinations
month,,,
2020-01,8140.0,254.0,0.000000e+00
2020-02,304998.0,9643.0,0.000000e+00
2020-03,2775012.0,169349.0,0.000000e+00
2020-04,8618449.0,783274.0,0.000000e+00
2020-05,12841220.0,762141.0,0.000000e+00
2020-06,15852831.0,582757.0,0.000000e+00
2020-07,24432576.0,652707.0,0.000000e+00
2020-08,37569280.0,882421.0,0.000000e+00
2020-09,33477707.0,647771.0,0.000000e+00


# VACUNADOS POR SEMESTRE

In [92]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Copia independiente del DataFrame original
conf_deaths_df = conf_deaths_df.copy()

# Convertir a datetime
conf_deaths_df.loc[:, 'date_format'] = pd.to_datetime(conf_deaths_df['date'])

# Crear una nueva columna para el semestre
conf_deaths_df.loc[:, 'semestre'] = (conf_deaths_df['date_format'].dt.year.astype(str) + ' - ' + ((conf_deaths_df['date_format'].dt.month - 1) // 6 + 1).astype(str))

# Agrupar por semestre y sumar los valores de 'new_vaccinations' y 'new_deaths'
grouped_semestre_df = conf_deaths_df.groupby('semestre').agg({'new_vaccinations': 'sum'}).reset_index()

# Asegúrate de que 'semestre' sea un string
grouped_semestre_df['semestre'] = grouped_semestre_df['semestre'].astype(str)

# Crear la figura
fig = go.Figure()

# Añadir gráfico de scatter para 'new_vaccinations'
fig.add_trace(go.Scatter(
    x=grouped_semestre_df['semestre'],
    y=grouped_semestre_df['new_vaccinations'],
    name='Vacunados',
    mode='lines+markers+text',  # Mostrar los valores en las líneas
    marker=dict(color='green', size=10, symbol='circle'),
    line=dict(color='green', width=2, dash='dashdot'),
    text=grouped_semestre_df['new_vaccinations'].apply(lambda x: f'{x / 1_000_000_000:,.2f}B'),  # Cambiar a miles de millones
    textposition='top center',
    textfont=dict(size=14, color='black')
))

# Añadir etiquetas, título y configurar la leyenda
fig.update_layout(
    title=dict(
        text='Gráfico de Vacunados - Covid 2019',
        font=dict(size=24, color='black'),
        x=0.5,  # Centrar el título
        xanchor='center'
    ),
    xaxis=dict(
        title='Vacunados x Semestre',
        title_standoff=30,
        tickangle=-45,
        showgrid=True,
        gridcolor='lightgrey',
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=16, color='black'),
        linecolor='black',
        linewidth=1
    ),
    yaxis=dict(
        title='Conteo (miles de millones)',  # Etiqueta del eje y
        range=[0, 30*1_000_000_000],  # Rango del eje Y en miles de millones
        tickvals=[i for i in range(0, 31*1_000_000_000, 5*1_000_000_000)],  # Intervalos de 5 mil millones en el eje Y
        ticktext=[f'{i/1_000_000_000}B' for i in range(0, 31*1_000_000_000, 5*1_000_000_000)],  # Texto de los ticks
        showgrid=True,
        gridcolor='lightgrey',
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=16, color='black'),
        linecolor='black',
        linewidth=1
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="black"
        ),
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='black',
        borderwidth=1
    ),
    plot_bgcolor='white',  # Fondo blanco para el gráfico
    paper_bgcolor='whitesmoke'  # Fondo blanco para el papel
)

# Mostrar el gráfico
fig.show()

# CASOS CONFIRMADOS

Plotly

In [82]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Copia independiente del DataFrame original
conf_deaths_df = conf_deaths_df.copy()

# Convertir a datetime
conf_deaths_df.loc[:, 'date_format'] = pd.to_datetime(conf_deaths_df['date'])

# Crear una nueva columna para el semestre
conf_deaths_df.loc[:, 'semestre'] = (conf_deaths_df['date_format'].dt.year.astype(str) + '-S' + ((conf_deaths_df['date_format'].dt.month - 1) // 6 + 1).astype(str))

# Agrupar por semestre y sumar los valores de 'new_cases' y 'new_deaths'
grouped_semestre_df = conf_deaths_df.groupby('semestre').agg({'new_cases': 'sum'}).reset_index()

# Asegúrate de que 'semestre' sea un string
grouped_semestre_df['semestre'] = grouped_semestre_df['semestre'].astype(str)

# Crear la figura
fig = go.Figure()

# Añadir gráfico de scatter para 'new_cases'
fig.add_trace(go.Scatter(
    x=grouped_semestre_df['semestre'],
    y=grouped_semestre_df['new_cases'],
    name='Confirmados',
    mode='lines+markers+text',  # Mostrar los valores en las líneas
    marker=dict(color='blue', size=10, symbol='circle'),
    line=dict(color='blue', width=2, dash='dashdot'),
    text=grouped_semestre_df['new_cases'].apply(lambda x: f'{x / 1_000_000_000:,.2f}B'),  # Cambiar a miles de millones
    textposition='top center',
    textfont=dict(size=14, color='black')
))

# Añadir etiquetas, título y configurar la leyenda
fig.update_layout(
    title=dict(
        text='Gráfico de Confirmados - Covid 2019',
        font=dict(size=24, color='black'),
        x=0.5,  # Centrar el título
        xanchor='center'
    ),
    xaxis=dict(
        title='Confirmados x Semestre',
        title_standoff=30,
        tickangle=-45,
        showgrid=True,
        gridcolor='lightgrey',
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=16, color='black'),
        linecolor='black',
        linewidth=1
    ),
    yaxis=dict(
        title='Conteo (miles de millones)',  # Etiqueta del eje y
        range=[0, 2*1_000_000_000],  # Rango del eje Y en miles de millones
        #tickvals=[i for i in range(0, 2*1_000_000_000, 1*1_000_000_000)],  # Intervalos de 1 mil millones en el eje Y
        #ticktext=[f'{i/1_000_000_000}B' for i in range(0, 11*1_000_000_000, 1*1_000_000_000)],  # Texto de los ticks
        showgrid=True,
        gridcolor='lightgrey',
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=16, color='black'),
        linecolor='black',
        linewidth=1
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="black"
        ),
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='black',
        borderwidth=1
    ),
    plot_bgcolor='white',  # Fondo blanco para el gráfico
    paper_bgcolor='whitesmoke'  # Fondo blanco para el papel
)

# Mostrar el gráfico
fig.show()

# CASOSO MUERTES

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Copia independiente del DataFrame original
conf_deaths_df = conf_deaths_df.copy()

# Convertir a datetime
conf_deaths_df.loc[:, 'date_format'] = pd.to_datetime(conf_deaths_df['date'])

# Crear una nueva columna para el semestre
conf_deaths_df.loc[:, 'semestre'] = (conf_deaths_df['date_format'].dt.year.astype(str) + '-S' + ((conf_deaths_df['date_format'].dt.month - 1) // 6 + 1).astype(str))

# Agrupar por semestre y sumar los valores de 'new_deaths' y 'new_deaths'
grouped_semestre_df = conf_deaths_df.groupby('semestre').agg({'new_deaths': 'sum'}).reset_index()

# Asegúrate de que 'semestre' sea un string
grouped_semestre_df['semestre'] = grouped_semestre_df['semestre'].astype(str)

# Crear la figura
fig = go.Figure()

# Añadir gráfico de scatter para 'new_deaths'
fig.add_trace(go.Scatter(
    x=grouped_semestre_df['semestre'],
    y=grouped_semestre_df['new_deaths'],
    name='Muertes',
    mode='lines+markers+text',  # Mostrar los valores en las líneas
    marker=dict(color='red', size=10, symbol='circle'),
    line=dict(color='red', width=2, dash='dashdot'),
    text=grouped_semestre_df['new_deaths'].apply(lambda x: f'{x / 1_000_000:,.2f}M'),
    textposition='top center',
    textfont=dict(size=14, color='black')
))

# Añadir etiquetas, título y configurar la leyenda
fig.update_layout(
    title=dict(
        text='Gráfico de Muertes - Covid 2019',
        font=dict(size=24, color='black'),
        x=0.5,  # Centrar el título
        xanchor='center'
    ),
    xaxis=dict(
        title='Muertes x Semestre',
        title_standoff=30,
        tickangle=-45,
        showgrid=True,
        gridcolor='lightgrey',
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=16, color='black'),
        linecolor='black',
        linewidth=1
    ),
    yaxis=dict(
        title='Conteo (millones)',  # Etiqueta del eje y
        range=[0, 10*1_000_000],  # Rango del eje Y en millones
        tickvals=[i for i in range(0, 11*1_000_000, 1*1_000_000)],  # Intervalos de 1 millón en el eje Y
        ticktext=[f'{i/1_000_000}M' for i in range(0, 11*1_000_000, 1*1_000_000)],  # Texto de los ticks
        showgrid=True,
        gridcolor='lightgrey',
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=16, color='black'),
        linecolor='black',
        linewidth=1
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="black"
        ),
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='black',
        borderwidth=1
    ),
    plot_bgcolor='white',  # Fondo blanco para el gráfico
    paper_bgcolor='whitesmoke'  # Fondo blanco para el papel
)

# Mostrar el gráfico
fig.show()
